# Code comparing different models and imputation methods for different missingness mechanisms.

In [20]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from scipy import stats

Baseline values.

In [4]:
# To obtain the measure of the variance below, run in R:
# > results <- make_data3bis(dim=9, size=200000)
# > var(results$y)

y_variances = {
    'mcar': 33,#10,
    'mnar': 33,
    'pred': 35,#10,
    'linearlinear': 25.4,
    'linearnonlinear': 1710,
    'nonlinearnonlinear': 1082,
}

### Compare different imputation methods across all models in the MCAR missingness mechanism.

First train a linear model for comparison

Read the csv file containing the data.

In [5]:
data = pd.read_csv(f'results/scores_mcar.csv', header=1,
                    names=['index', 'score', 'method', 'forest'])

# Knowing the variance of y, we can extract the R2
data['R2'] = 1 - data['score'] / y_variances['mcar']
# The fold number is encoded at the end of the name of the index
data['fold'] = data['index'].str.extract('(\d+)$').astype(int)

data

,index,score,method,forest,R2,fold
0,rpart2,9.848399,rpart,DECISION TREE,0.701564,2
1,rpart3,10.199162,rpart,DECISION TREE,0.690934,3
2,rpart4,9.158413,rpart,DECISION TREE,0.722472,4
3,rpart5,10.289807,rpart,DECISION TREE,0.688188,5
4,rpart6,10.183333,rpart,DECISION TREE,0.691414,6
...,...,...,...,...,...,...
3694,Gaussian + mask (knn)96,5.401649,Gaussian + mask,KNN,0.836314,96
3695,Gaussian + mask (knn)97,6.290848,Gaussian + mask,KNN,0.809368,97
3696,Gaussian + mask (knn)98,5.523105,Gaussian + mask,KNN,0.832633,98
3697,Gaussian + mask (knn)99,6.218253,Gaussian + mask,KNN,0.811568,99


In [19]:
# find number of distinct categories in method and fold
print('method categories:', len(set(data['method'])))
print('fold categories:', len(set(data['fold'])))

method categories: 11
fold categories: 100


In [11]:
X = data[['method', 'fold']]
y = data['R2']

In [12]:
# Create a ColumnTransformer to handle categorical variables
preprocessor = ColumnTransformer(
    transformers=[
        ('method', OneHotEncoder(), ['method']),
        ('simu', OneHotEncoder(), ['fold'])
    ],
    remainder='passthrough'
)

In [13]:
# Create a pipeline with the preprocessor and linear regression model
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [14]:
# Train the model on the training data
model.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('method', OneHotEncoder(),
                                                  ['method']),
                                                 ('simu', OneHotEncoder(),
                                                  ['fold'])])),
                ('regressor', LinearRegression())])

In [17]:
# Print coefficients
coefficients = model.named_steps['regressor'].coef_
print('Coefficients:', coefficients)

Coefficients: [ 8.49055297e-02  8.42033945e-02  5.04157850e-02 -5.76495619e-02
 -5.51604165e-02  4.74716128e-02  5.80045590e-02 -6.13315981e-02
 -5.97858254e-02 -4.57080449e-02 -4.53654342e-02 -1.44096840e-02
 -5.62759620e-03 -2.10639596e-02  2.17827680e-02 -4.46910650e-03
 -1.64025014e-02 -1.69824850e-02 -7.97692429e-04 -2.19236711e-02
  9.50242259e-03 -2.69044431e-03  6.85077580e-03 -1.25677006e-02
  1.72706079e-02  1.98399494e-02 -1.42645449e-03  3.40854952e-02
  1.46697871e-03 -1.52478376e-02 -7.62688710e-02 -2.20523469e-02
  3.15775702e-02  1.61894722e-02  2.70271068e-02  2.69742547e-02
 -3.58331831e-03  1.07979912e-02  8.44659506e-03  2.24496869e-02
 -3.30479804e-02 -5.86708651e-06 -2.44312680e-02  3.31072393e-02
 -1.23802751e-02  1.48498826e-02  1.05727935e-02 -1.27204505e-02
 -2.50699656e-04  1.03741233e-02  2.12283782e-03  2.73970372e-02
 -3.38413772e-02  3.05183108e-02 -6.53934731e-03 -7.72537154e-03
 -9.56288173e-03 -1.73093031e-03 -3.47199072e-02 -4.41017420e-03
 -1.1646698

Perform paired t-tests between each pair of methods. Save the p-values into a matrix.

In [43]:
data

,index,score,method,forest,R2,fold
0,rpart2,9.848399,rpart,DECISION TREE,0.701564,2
1,rpart3,10.199162,rpart,DECISION TREE,0.690934,3
2,rpart4,9.158413,rpart,DECISION TREE,0.722472,4
3,rpart5,10.289807,rpart,DECISION TREE,0.688188,5
4,rpart6,10.183333,rpart,DECISION TREE,0.691414,6
...,...,...,...,...,...,...
3694,Gaussian + mask (knn)96,5.401649,Gaussian + mask,KNN,0.836314,96
3695,Gaussian + mask (knn)97,6.290848,Gaussian + mask,KNN,0.809368,97
3696,Gaussian + mask (knn)98,5.523105,Gaussian + mask,KNN,0.832633,98
3697,Gaussian + mask (knn)99,6.218253,Gaussian + mask,KNN,0.811568,99


In [38]:
# find the unique categories in method
method_categories = set(data['method'])
print('method categories:', method_categories)
method_categories = list(method_categories)
method_categories.remove('rpart')
method_categories.remove('rpart + mask')
method_categories.remove('ctree')
method_categories.remove('ctree + mask')
method_categories.remove('MIA')
print(len(method_categories))

method categories: {'mean', 'rpart + mask', 'oor + mask', 'ctree', 'oor', 'ctree + mask', 'MIA', 'rpart', 'Gaussian + mask', 'mean + mask', 'Gaussian'}
6


In [46]:
pval_matrix = []
for i in range(len(method_categories)):
    for j in range(i+1, len(method_categories)):
        # compare method i and method j by extracting their respective R2 values and
        # using a paired t test to compare them
        method_i = data[data['method'] == method_categories[i]]['R2']
        print(method_categories[i], np.mean(method_i), np.std(method_i))

        method_j = data[data['method'] == method_categories[j]]['R2']
        print(method_categories[j], np.mean(method_j), np.std(method_j))

        # calculate the test statistic
        t_statistic, pval = stats.ttest_rel(method_i, method_j)
        # print out the test if the p-value is less than alpha = 0.05
        if pval < 0.05:
            print(method_categories[i], 'and', method_categories[j], 'are statistically significant, pval=', pval)
        print()

mean 0.7978076942751718 0.06624989399002952
oor + mask 0.6905502560383827 0.13374050857417197
mean and oor + mask are statistically significant, pval= 2.7784036373532677e-66

mean 0.7978076942751718 0.06624989399002952
oor 0.689004483319343 0.13354277258876096
mean and oor are statistically significant, pval= 2.571473395747939e-67

mean 0.7978076942751718 0.06624989399002952
Gaussian + mask 0.8345394759028779 0.05343602018695556
mean and Gaussian + mask are statistically significant, pval= 1.1226449816946505e-143

mean 0.7978076942751718 0.06624989399002952
mean + mask 0.8083406404777675 0.06977655336808952
mean and mean + mask are statistically significant, pval= 2.4629355746438794e-43

mean 0.7978076942751718 0.06624989399002952
Gaussian 0.8352416111116086 0.05491263473692301
mean and Gaussian are statistically significant, pval= 2.2877524658249477e-153

oor + mask 0.6905502560383827 0.13374050857417197
oor 0.689004483319343 0.13354277258876096
oor + mask and oor are statistically si